# Task Number 5

## Yuval Bar - 206011355

## Gal Giller - 209264555

In [69]:
import requests
import pandas as pd
data = pd.read_csv("destinations_LP_crawler_Ex5.csv")
LP_destinations = pd.DataFrame(data["city"])
country =  []
api_key = ''
try:
    for i in data["city"]:
        address = i
        url_country = 'https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s' % (address,api_key)
        try:
            response = requests.get(url_country)
            if not response.status_code==200:
                print("HTTP error",response.status_code)
            else:
                try:
                    response = requests.get(url_country).json()
                    country_is = response['results'][0]['formatted_address'].split(",")[-1].rstrip().lstrip()
                    contains_digit = False
                    for character in country_is:
                        if character.isdigit():
                            contains_digit = True
                    if contains_digit == False:
                        country.append(country_is)
                    else:
                        country_is = response['results'][0]['formatted_address'].split(",")[-2].rstrip().lstrip()
                        country.append(country_is)
                except:
                    print("response not in valid JSON format")
        except:
            print("Something went wrong with requests.get")
except:
    print("Something went wrong")
LP_destinations["Country"] = country

In [79]:
from bs4 import BeautifulSoup
description = []
j = 0
for i in data["city_LP"]:
    url = "https://www.lonelyplanet.com/" + country[j] + "/" + i
    j = j + 1
    response = requests.get(url)
    if not response.status_code==200:
        print("HTTP error",response.status_code)
    try:
        results_page = BeautifulSoup(response.content,'html.parser')
        try:
            descrip = results_page.find('section',class_='text-black-400')
            descr = descrip.find('div')
            string = ""
            for tag in descr.find_all():
                if tag.name == 'p':
                    string = string + tag.get_text()
                elif tag.name =='h2':
                    string = string + "\n" + tag.get_text() + ": "
                else:
                    continue
        except:
            descr = ''
        description.append(string)
    except:
        print("Something went wrong")

LP_destinations["Description"] = description
LP_destinations

,city,Country,Description
0,Antalya,Turkey,Once seen simply as the gateway to the Turkish...
1,Athens,Greece,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...,...
71,Venice,Italy,Imagine the audacity of building a city of mar...
72,Vienna,Austria,Baroque streetscapes and imperial palaces set ...
73,Warsaw,Poland,"A phoenix arisen from the ashes, Poland's capi..."
74,Zakynthos,Greece,"Zakynthos, also known by its Italian name Zant..."


In [80]:
LP_destinations.to_csv('LP_destinations.csv', index=False)